In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import boto3

%matplotlib inline

In [2]:
BUCKET = "mlend-capstone"
FILENAME = "preproc_data.csv" #Preprocessed data from the previous notebook

In [3]:
s3 = boto3.resource('s3')
obj = s3.Object(BUCKET, FILENAME)
body = obj.get()['Body'].read().decode('utf-8')

In [4]:
contents = body.split('\n')

In [5]:
data = pd.DataFrame((row.split(',') for row in contents[1:]), columns=contents[0].split(','))

data.tail()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,NoShow
110523,3596266328735,5650093,F,2016-05-03 07:27:33,2016-06-07 00:00:00,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,15576631729893,5630692,F,2016-04-27 16:03:52,2016-06-07 00:00:00,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,92134931435557,5630323,F,2016-04-27 15:09:23,2016-06-07 00:00:00,38,MARIA ORTIZ,0,0,0,0,0,1,No
110526,377511518121127,5629448,F,2016-04-27 13:30:56,2016-06-07 00:00:00,54,MARIA ORTIZ,0,0,0,0,0,1,No
110527,,None,None,None,None,None,None,None,None,None,None,None,None,None


In [6]:
#Drop the trailing row
data = data.drop(index=[110527])

In [7]:
#Check for duplicates
data.duplicated().sum()

0

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null object
AppointmentID     110527 non-null object
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null object
Neighbourhood     110527 non-null object
Scholarship       110527 non-null object
Hypertension      110527 non-null object
Diabetes          110527 non-null object
Alcoholism        110527 non-null object
Handicap          110527 non-null object
SMS_received      110527 non-null object
NoShow            110527 non-null object
dtypes: object(14)
memory usage: 12.6+ MB


In [9]:
#We apply the same data types from the previous notebook
data_type_map = {'PatientId': 'str',
                 'AppointmentID': 'str',
                 'Gender': 'category',
                 'ScheduledDay': 'datetime64[ns]',
                 'AppointmentDay': 'datetime64[ns]',
                 'Age': 'int',
                 'Neighbourhood': 'category',
                 'Scholarship': 'category',
                 'Hypertension': 'category',
                 'Diabetes': 'category',
                 'Alcoholism': 'category',
                 'Handicap': 'category',
                 'SMS_received': 'category',
                 'NoShow': 'int'
                }

data.NoShow = data.NoShow.map({'No':0, 'Yes':1})
data = data.astype(dtype=data_type_map)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null object
AppointmentID     110527 non-null object
Gender            110527 non-null category
ScheduledDay      110527 non-null datetime64[ns]
AppointmentDay    110527 non-null datetime64[ns]
Age               110527 non-null int64
Neighbourhood     110527 non-null category
Scholarship       110527 non-null category
Hypertension      110527 non-null category
Diabetes          110527 non-null category
Alcoholism        110527 non-null category
Handicap          110527 non-null category
SMS_received      110527 non-null category
NoShow            110527 non-null int64
dtypes: category(8), datetime64[ns](2), int64(2), object(2)
memory usage: 6.7+ MB


In [10]:
"""
We will add features:
    prev_visits:= number of previous scheduled visits for a given patient
    prve_noshows:= number of times the patient has not shown up for appointments
    pct_noshow:= prev_noshows / prev_visits
    appt_day:= day of the week the appointment is scheduled (Monday, Tuesday, etc.)
    days_wait:= number of days between when the appointment was scheduled and when it was scheduled to occur
"""
data['prev_visits'] = data.groupby('PatientId')['Gender'].transform('size')
data['prev_noshows'] = data.groupby('PatientId')['NoShow'].transform('sum')
data['pct_noshows'] = data.prev_noshows / data.prev_visits
data['appt_day'] = data.AppointmentDay.apply(lambda x: x.weekday()).astype('category')
data['days_wait'] = data.AppointmentDay - data.ScheduledDay
data['days_wait'] = data.days_wait.apply(lambda x: -x.days)

data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,NoShow,prev_visits,prev_noshows,pct_noshows,appt_day,days_wait
0,29872499824296,5642903,F,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0,2,0,0.0,4,1
1,558997776694438,5642503,M,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0,2,0,0.0,4,1
2,4262962299951,5642549,F,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0,2,0,0.0,4,1
3,867951213174,5642828,F,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,2,1,0.5,4,1
4,8841186448183,5642494,F,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0,1,0,0.0,4,1


In [11]:
data[['prev_visits', 'prev_noshows', 'pct_noshows', 'days_wait']].describe()

,prev_visits,prev_noshows,pct_noshows,days_wait
count,110527.000000,110527.000000,110527.000000,110527.000000
mean,3.540628,0.632796,0.201933,-9.183702
std,6.562050,1.145807,0.314005,15.254996
min,1.000000,0.000000,0.000000,-178.000000
25%,1.000000,0.000000,0.000000,-14.000000
50%,2.000000,0.000000,0.000000,-3.000000
75%,4.000000,1.000000,0.333333,1.000000
max,88.000000,18.000000,1.000000,7.000000


In [12]:
data.isna().sum()

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hypertension      0
Diabetes          0
Alcoholism        0
Handicap          0
SMS_received      0
NoShow            0
prev_visits       0
prev_noshows      0
pct_noshows       0
appt_day          0
days_wait         0
dtype: int64

In [13]:
#We drop columns that might introduce bias (Gender and Neighbourhood), no useful info (Ids), and datetime objects 
cols = ['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood']

In [14]:
X = data.drop(columns=['NoShow']+cols)
y = data.NoShow

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110527 entries, 0 to 110526
Data columns (total 12 columns):
Age             110527 non-null int64
Scholarship     110527 non-null category
Hypertension    110527 non-null category
Diabetes        110527 non-null category
Alcoholism      110527 non-null category
Handicap        110527 non-null category
SMS_received    110527 non-null category
prev_visits     110527 non-null int64
prev_noshows    110527 non-null int64
pct_noshows     110527 non-null float64
appt_day        110527 non-null category
days_wait       110527 non-null int64
dtypes: category(7), float64(1), int64(4)
memory usage: 5.8 MB


In [16]:
categorical_cols = []

for name in X.columns:
    if isinstance(X[name].dtype, pd.core.dtypes.dtypes.CategoricalDtype):
        categorical_cols.append(name)
        
print(categorical_cols)

['Scholarship', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received', 'appt_day']


In [17]:
from sklearn.preprocessing import StandardScaler

X_cont = X.drop(columns=categorical_cols)

scaler = StandardScaler()
scaler.fit(X_cont)

X_cont_scaled = pd.DataFrame(scaler.transform(X_cont), columns=X_cont.columns)

In [18]:
#We'll need these for the final application
print("Scale: ", scaler.scale_)
print("Mean: ", scaler.mean_)
print("Variance: ", scaler.var_)

Scale:  [23.11010042  6.56202063  1.14580168  0.31400402 15.25492721]
Mean:  [37.08887421  3.54062808  0.63279561  0.20193256 -9.18370172]
Variance:  [5.34076741e+02 4.30601148e+01 1.31286150e+00 9.85985238e-02
 2.32712804e+02]


In [19]:
X_cat = data[categorical_cols+["NoShow"]]

In [20]:
X_prepped = X_cont_scaled.merge(X_cat, left_index=True, right_index=True)

In [21]:
X_prepped.head()

,Age,prev_visits,prev_noshows,pct_noshows,days_wait,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,appt_day,NoShow
0,1.077932,-0.234780,-0.552273,-0.643089,0.667568,0,1,0,0,0,0,4,0
1,0.818306,-0.234780,-0.552273,-0.643089,0.667568,0,0,0,0,0,0,4,0
2,1.077932,-0.234780,-0.552273,-0.643089,0.667568,0,0,0,0,0,0,4,0
3,-1.258708,-0.234780,0.320478,0.949247,0.667568,0,0,0,0,0,0,4,0
4,0.818306,-0.387172,-0.552273,-0.643089,0.667568,0,1,1,0,0,0,4,0


In [22]:
#Save prepped data
from io import StringIO

csv_buffer = StringIO()
X_prepped.to_csv(csv_buffer, header=True, index=False)
obj = s3.Object(BUCKET, 'engineered_data.csv')
obj.put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '3B7015DC1C41ECF5',
  'HostId': 'E3zrC5bZw/XDuuztxkcyryp0A93ij8PmOIrbAJopl3LoL1t4yxme3wrAJZoIrRr8M0PX6e9cpF0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'E3zrC5bZw/XDuuztxkcyryp0A93ij8PmOIrbAJopl3LoL1t4yxme3wrAJZoIrRr8M0PX6e9cpF0=',
   'x-amz-request-id': '3B7015DC1C41ECF5',
   'date': 'Wed, 22 Apr 2020 19:38:33 GMT',
   'etag': '"fd4890ce58a9e9dc7de034cf9f253ec1"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"fd4890ce58a9e9dc7de034cf9f253ec1"'}

In [23]:
obj = s3.Object(BUCKET, 'scales')
obj.put(Body=scaler.scale_.tobytes())
obj = s3.Object(BUCKET, 'means')
obj.put(Body=scaler.mean_.tobytes())
obj = s3.Object(BUCKET, 'variances')
obj.put(Body=scaler.var_.tobytes())

{'ResponseMetadata': {'RequestId': '7E9FB019FD4D8C87',
  'HostId': 'ghxUfvm367rNyx/MpPW4TUeiFsXjoaeEYfhKsielcfIADdkTMWimobg+susnxB96g70FvKZotiQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ghxUfvm367rNyx/MpPW4TUeiFsXjoaeEYfhKsielcfIADdkTMWimobg+susnxB96g70FvKZotiQ=',
   'x-amz-request-id': '7E9FB019FD4D8C87',
   'date': 'Wed, 22 Apr 2020 19:38:33 GMT',
   'etag': '"dd487a311e899430e0c92c78246aea4e"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"dd487a311e899430e0c92c78246aea4e"'}

In [24]:
#Check that the data was properly saved
from io import BytesIO

obj = boto3.client('s3').get_object(Bucket=BUCKET, Key='engineered_data.csv')
data_check = pd.read_csv(BytesIO(obj['Body'].read()), index_col=False)

In [25]:
data_check.head()

,Age,prev_visits,prev_noshows,pct_noshows,days_wait,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,appt_day,NoShow
0,1.077932,-0.234780,-0.552273,-0.643089,0.667568,0,1,0,0,0,0,4,0
1,0.818306,-0.234780,-0.552273,-0.643089,0.667568,0,0,0,0,0,0,4,0
2,1.077932,-0.234780,-0.552273,-0.643089,0.667568,0,0,0,0,0,0,4,0
3,-1.258708,-0.234780,0.320478,0.949247,0.667568,0,0,0,0,0,0,4,0
4,0.818306,-0.387172,-0.552273,-0.643089,0.667568,0,1,1,0,0,0,4,0


In [26]:
data_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 13 columns):
Age             110527 non-null float64
prev_visits     110527 non-null float64
prev_noshows    110527 non-null float64
pct_noshows     110527 non-null float64
days_wait       110527 non-null float64
Scholarship     110527 non-null int64
Hypertension    110527 non-null int64
Diabetes        110527 non-null int64
Alcoholism      110527 non-null int64
Handicap        110527 non-null int64
SMS_received    110527 non-null int64
appt_day        110527 non-null int64
NoShow          110527 non-null int64
dtypes: float64(5), int64(8)
memory usage: 11.0 MB
